# Train Merlin TwoTower model

### Notebook Steps
* Build custom Vertex training container based on NVIDIA NGC Merlin Training container
* Confiruger and submit Vertec custom training job
* Configure and submit hyperparameter tuning job
* Evaluate results of hyperparameter tuning job

### Negative Sampling

* Merlin provides scalable negative sampling algorithms for the Item Retrieval Task 
* In this example, the in-batch sampling algorithm, which uses the items interacted by other users as negatives within the same mini-batch

### Setup

In [2]:
import json
import os
import time

from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform import hyperparameter_tuning as hpt

In [3]:
# TODO: Project definitions
PROJECT_ID = 'hybrid-vertex' # Change to your project ID.
REGION = 'us-central1' # Change to your region.

# TODO: Service Account address
VERTEX_SA = '934903580331-compute@developer.gserviceaccount.com' # Change to your service account with Vertex AI Admin permitions.

### For HugeCTR data access

* must be a `/gcs/BUCKET_NAME/...` path for GCSFuse 

In [4]:
# using GCSFuse file lists
TRAIN_DATA = '/gcs/spotify-merlin-v1/nvt-preprocessing-spotify-v10-subset/nvt-processed/train/_gcs_file_list.txt'
VALID_DATA = '/gcs/spotify-merlin-v1/nvt-preprocessing-spotify-v10-subset/nvt-processed/valid/_gcs_file_list.txt'

# Schema used by the training pipepine
SCHEMA_PATH = '/gcs/spotify-merlin-v1/nvt-preprocessing-spotify-v10-subset/nvt-processed/train/schema.pbtxt'

# Merline Datatsets
# train = MerlinDataset(output_train_dir + "/*.parquet", schema=schema, part_size="500MB")
# valid = MerlinDataset(output_valid_dir + "/*.parquet", schema=schema, part_size="500MB")

In [ ]:
# Bucket definitions
BUCKET = 'spotify-merlin-v1'

VERSION = 'v01-subset'
MODEL_NAME = 'TwoTower'
MODEL_DISPLAY_NAME = f'hugectr-{MODEL_NAME}-{VERSION}'
WORKSPACE = f'gs://{BUCKET}/{MODEL_DISPLAY_NAME}'

# Docker definitions for training
IMAGE_NAME = f'hugectr-{MODEL_NAME}-training-{VERSION}'
IMAGE_URI = f'gcr.io/{PROJECT_ID}/{IMAGE_NAME}'
DOCKERNAME = 'hugectr'


### Initialize Vertex AI SDK

In [ ]:
vertex_ai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=os.path.join(WORKSPACE, 'stg')
)

### Submit a Vertex custom training job